In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import json
import numpy as np
import os
import random
import scipy.io as sio
import torch
import torch.nn as nn
#from torchsummary import summary
STEP =256

In [2]:
#Padded input data , labels are arrays which are also padded to maximum size , 
#its not one hot encoded. 
data = np.load('train.npz')
train_x = data['a']
train_y = data['b']
print(train_x.shape,train_y.shape)

(7676, 18176, 1) (7676, 71)


In [3]:
#just an generator for training.
def data_generator(batch_size, x, y):
    num_examples = len(x)
    examples = zip(x, y)
    examples = sorted(examples, key = lambda x: x[0].shape[0])
    end = num_examples - batch_size + 1
    batches = [examples[i:i+batch_size]
                for i in range(0, end, batch_size)]
    random.shuffle(batches)
    batchlen = int(len(batches))
  
    i =0
    while True:

        if i >= batchlen:
               return
        #print(i)
        x, y = zip(*batches[i])
        yield np.asarray(x), np.asarray(y)
        i = i +1
        
            #yield x,y

In [5]:
class Bn_Relu(nn.Module):
    def __init__(self, output,dropout=0):
        super(Bn_Relu,self).__init__()
        self.bn =  nn.BatchNorm1d(output)
        self.relu = nn.ReLU()
        self.drop = None
        if dropout > 0:
           self.drop = nn.Dropout(dropout)
    def forward(self,x):
        out = self.bn(x)
        out = self.relu(out)
        if self.drop is not None:
            out = self.drop(out)
        return out

In [6]:
def get_padding(requiredSize, originalSize):
    x = requiredSize - originalSize
    if x % 2 == 0:
        return (int(x/2) ,int(x/2))
    else:
        x = int(x/2)
        return (x+1,x)

In [7]:
class ResnetBlock(nn.Module):
    def __init__(self,prev_filters,num_filters,subsample_length,block_index,dropout):
        super(ResnetBlock, self).__init__()
        self.subsample_length = subsample_length
        self.max1 = nn.MaxPool1d(kernel_size=subsample_length)
        self.zero_pad = (block_index % 4) == 0 and block_index > 0
        #if zero_pad is True:
            #shortcut = Lambda(zeropad, output_shape=zeropad_output_shape)(shortcut)
        print("Conv 1" ,prev_filters,num_filters,subsample_length)
        self.conv1 = nn.Conv1d(prev_filters, num_filters, 16, subsample_length)
        self.bn1 = None
        if(block_index !=0):
              self.bn1 = Bn_Relu(num_filters, dropout )
        print("Conv 2" ,num_filters)
        self.conv2 = nn.Conv1d(num_filters, num_filters,16, 1)
    
    def forward(self,x):
        iden = self.max1(x)
                       
              
        out = self.conv1(x)
        if self.bn1 is not None:
            out = self.bn1(out)
        out = self.conv2(out)
        #print ("out Before ",out.shape,iden.shape)
        if(iden.shape[2] != out.shape[2]):
            #convSize = ((out.shape[2] - (16-1)-1)/self.subsample_length) +1
            #convSize = int(convSize)
            p1d = get_padding(iden.shape[2],out.shape[2])
            #print ("paddings",p1d)
            out = F.pad(out, p1d, "constant", 0) 
            #print ("out After",out.shape[2])
        if(iden.shape[1] < out.shape[1]):
            #print("shape of middle",out.shape[1],iden.shape[1])
            xpad = (0,0)
            xpad = xpad + get_padding(out.shape[1],iden.shape[1])
            #print ("x paddings",xpad)
            iden = F.pad(iden, xpad, "constant", 0) 
            #print ("iden After",iden.shape[1])
           
        out += iden
        return out       

In [8]:
def get_num_filters_at_index(index, num_start_filters):
    return 2**int(index / 4) \
        * num_start_filters

In [9]:
class ResNet(nn.Module):

    def __init__(self,dropout):
        super(ResNet, self).__init__()
        self.layers1 = nn.Conv1d(1, 32, 16, 1)
        self.bn1 = Bn_Relu(32)
        layers = []
        res = [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
        #res = [1,2,1]
        prev_filter =32
        self.debug =0;
        for index, subsample_length in enumerate(res):
            num_filters = get_num_filters_at_index(index, 32 )
            print("filter length",prev_filter,num_filters)
            x =  ResnetBlock(prev_filter,num_filters,subsample_length,index,dropout)
            y = Bn_Relu(num_filters,dropout)
            prev_filter = num_filters
            layers.append(x)
            layers.append(y)
        self.resLayer = nn.Sequential(*layers)
        self.linear1 = nn.Linear(256,4)
        
    def forward(self, x):
        #print(self.debug)
        self.debug +=1
        x= self.layers1(x)
        x= self.bn1(x)
        x = self.resLayer(x)
        x = F.pad(x, (0,1), "constant", 0) 
        x = x.permute(0,2,1)
        
        x = self.linear1(x)
        #y = x.view(x.size()[0], -1);
        #print ("final shape ", x.shape)
        #return F.log_softmax(self.linear1(x.view(x.size()[0], -1)), dim=1)
        return F.log_softmax(x, dim=1)
        #return x
        
        

In [10]:
import torch.nn.functional as F
loss_fn = nn.CrossEntropyLoss(size_average=False)

C:\Users\rajee\AppData\Local\Continuum\anaconda3\envs\fresh\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [11]:
model = ResNet(dropout =0.2)

filter length 32 32
Conv 1 32 32 1
Conv 2 32
filter length 32 32
Conv 1 32 32 2
Conv 2 32
filter length 32 32
Conv 1 32 32 1
Conv 2 32
filter length 32 32
Conv 1 32 32 2
Conv 2 32
filter length 32 64
Conv 1 32 64 1
Conv 2 64
filter length 64 64
Conv 1 64 64 2
Conv 2 64
filter length 64 64
Conv 1 64 64 1
Conv 2 64
filter length 64 64
Conv 1 64 64 2
Conv 2 64
filter length 64 128
Conv 1 64 128 1
Conv 2 128
filter length 128 128
Conv 1 128 128 2
Conv 2 128
filter length 128 128
Conv 1 128 128 1
Conv 2 128
filter length 128 128
Conv 1 128 128 2
Conv 2 128
filter length 128 256
Conv 1 128 256 1
Conv 2 256
filter length 256 256
Conv 1 256 256 2
Conv 2 256
filter length 256 256
Conv 1 256 256 1
Conv 2 256
filter length 256 256
Conv 1 256 256 2
Conv 2 256


In [12]:
model.cuda()

ResNet(
  (layers1): Conv1d(1, 32, kernel_size=(16,), stride=(1,))
  (bn1): Bn_Relu(
    (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (resLayer): Sequential(
    (0): ResnetBlock(
      (max1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv1d(32, 32, kernel_size=(16,), stride=(1,))
      (conv2): Conv1d(32, 32, kernel_size=(16,), stride=(1,))
    )
    (1): Bn_Relu(
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (drop): Dropout(p=0.2)
    )
    (2): ResnetBlock(
      (max1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv1d(32, 32, kernel_size=(16,), stride=(2,))
      (bn1): Bn_Relu(
        (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (drop): Dropout(p=0.2)
      )
      (conv2): Con

In [13]:
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
def TrainModel(epoch,batchsize):
    model.train()    
    for epoch in range(epoch):
        print('*'*10)
        print('Epoch: {}'.format(epoch))
        tloss = 0.0
        taccu = 0.0
        train_gen = data_generator(batchsize,train_x,train_y)
        for x,y in train_gen:
            #print (x.shape,y.shape)
            inp = np.transpose(x, (0, 2, 1))
            inp =torch.from_numpy(inp).requires_grad_()
            inpTensor= inp.cuda()
            optimizer.zero_grad()
            outputs = model(inpTensor)
            labels = torch.from_numpy(y).long()
            labels = labels.cuda()
            loss = loss_fn(outputs.view(outputs.shape[0]*outputs.shape[1],-1), labels.view(labels.shape[0]*labels.shape[1]))
            tloss += loss.item()
            pred = torch.max(outputs, -1)[1]
            taccu += (pred == labels).sum().item()
            #print("loss ", loss)
            loss.backward()
            optimizer.step()
            
        print('Loss: {}'.format(tloss))
        print('Accu: {}'.format(taccu/len(train_x)))

In [15]:
def ValidateModel(batchsize):
    model.eval()
    tloss = 0.0
    taccu = 0.0
    valid_gen = data_generator(batchsize,valid_x,valid_y)
    for x,y in valid_gen:
        inp = np.transpose(x, (0, 2, 1))
        inp =torch.from_numpy(inp)
        inpTensor= inp.cuda()
        outputs = model(inpTensor)
        labels = torch.from_numpy(y).long()
        labels = labels.cuda()
        loss = loss_fn(outputs.view(outputs.shape[0]*outputs.shape[1],-1), labels.view(labels.shape[0]*labels.shape[1]))
        tloss += loss.item()
        pred = torch.max(outputs, -1)[1]
        taccu += (pred == labels).sum().item()
    print('Loss: {}'.format(tloss))
    print('Accu: {}'.format(taccu/len(valid_x)))

In [16]:
data = np.load('valid.npz')
valid_x = data['a']
valid_y = data['b']
print(valid_x.shape,valid_y.shape)

(852, 18176, 1) (852, 71)


In [17]:
model.load_state_dict(torch.load("test-1.pth"))

In [17]:
torch.cuda.empty_cache()
TrainModel(1,32)

**********
Epoch: 0
Loss: 377056.09240722656
Accu: 50.060838978634706


In [18]:
TrainModel(1,32)

**********
Epoch: 0
Loss: 271993.95275878906
Accu: 55.935513288170924


In [19]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 30984.485260009766
Accu: 54.40492957746479


In [19]:
torch.save(model.state_dict(), "test-1.pth")

In [20]:
torch.cuda.empty_cache()
TrainModel(5,16)

**********
Epoch: 0
Loss: 193934.66807556152
Accu: 60.43564356435643
**********
Epoch: 1
Loss: 190329.93565368652
Accu: 60.642391870766026
**********
Epoch: 2
Loss: 186438.39692687988
Accu: 60.667404898384575
**********
Epoch: 3
Loss: 179197.81770324707
Accu: 61.37428348097968
**********
Epoch: 4
Loss: 177761.2780456543
Accu: 61.36933298593017


In [21]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 19819.917793273926
Accu: 61.389671361502344


In [22]:
torch.save(model.state_dict(), "test-1.pth")

In [23]:
torch.cuda.empty_cache()
TrainModel(6,16)

**********
Epoch: 0
Loss: 170883.05847930908
Accu: 61.93720687858259
**********
Epoch: 1
Loss: 168821.39450073242
Accu: 61.99231370505471
**********
Epoch: 2
Loss: 163004.5175704956
Accu: 62.45909327774883
**********
Epoch: 3
Loss: 161507.6676864624
Accu: 62.4254820218864
**********
Epoch: 4
Loss: 155492.19296264648
Accu: 62.84158415841584
**********
Epoch: 5
Loss: 154064.56114196777
Accu: 62.794424179260034


In [24]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 17882.455078125
Accu: 62.082159624413144


In [25]:
torch.save(model.state_dict(), "test-1.pth")

In [26]:
torch.cuda.empty_cache()
TrainModel(6,16)

**********
Epoch: 0
Loss: 151394.92232513428
Accu: 62.98449713392392
**********
Epoch: 1
Loss: 148370.0378189087
Accu: 63.238535695674834
**********
Epoch: 2
Loss: 145475.99559020996
Accu: 63.44971339239187
**********
Epoch: 3
Loss: 143407.31893920898
Accu: 63.52579468473163
**********
Epoch: 4
Loss: 141174.9951095581
Accu: 63.601875977071394
**********
Epoch: 5
Loss: 137675.82921600342
Accu: 63.76341844710787


In [27]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 17813.987754821777
Accu: 62.35798122065728


In [28]:
torch.save(model.state_dict(), "test-2.pth")

In [29]:
model.load_state_dict(torch.load("test-2.pth"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [30]:
torch.cuda.empty_cache()
TrainModel(6,16)

**********
Epoch: 0
Loss: 136388.07900238037
Accu: 63.86099531005732
**********
Epoch: 1
Loss: 132161.95597076416
Accu: 64.10265763418447
**********
Epoch: 2
Loss: 131530.73462677002
Accu: 64.1323606044815
**********
Epoch: 3
Loss: 129756.62490463257
Accu: 64.34236581552892
**********
Epoch: 4
Loss: 126288.5089263916
Accu: 64.35330901511203
**********
Epoch: 5
Loss: 126232.15660095215
Accu: 64.3882230328296


In [31]:
torch.cuda.empty_cache()
ValidateModel(16)

Loss: 15539.197761535645
Accu: 63.617370892018776


In [32]:
torch.save(model.state_dict(), "test-2.pth")